In [1]:
# # Google colab connection
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/Dataset/RSBDSL38_train_val_test.zip"

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, DenseNet201, Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Define directories
train_val_dir = '/content/train_val_data'  # Replace with your path
test_dir = '/content/test_data'  # Replace with your path

# Hyperparameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = 38

# Create image data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

Found 8546 images belonging to 38 classes.
Found 2115 images belonging to 38 classes.
Found 1203 images belonging to 38 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, DenseNet121, ResNet50, DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# Function to build a single model
def build_model(base_model, model_name):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(38, activation='softmax')(x)  # Assuming 38 classes

    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load pre-trained models
xception_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
den201_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in xception_model.layers:
    layer.trainable = True

for layer in den201_model.layers:
    layer.trainable = True

for layer in resnet_model.layers:
    layer.trainable = True
# Build and compile each model separately
model1 = build_model(xception_model, model_name='Xception_Model')
model2 = build_model(den201_model, model_name='DenseNet201_Model')
model3 = build_model(resnet_model, model_name='ResNet50_Model')

# Optionally, you can print summaries of each model to inspect their architecture
model1.summary()
model2.summary()
model3.summary()

Model: "Xception_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 7, 7, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 38)                  │          38,950 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,998,606 (87.73 MB)

 Trainable params: 22,944,078 (87.52 MB)

 Non-trainable params: 54,528 (213.00 KB)

...

Model: "ResNet50_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 38)                  │          38,950 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,724,838 (98.13 MB)

 Trainable params: 25,671,718 (97.93 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [4]:
# # Define function to create model
# def create_model(base_model, num_classes):
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     outputs = Dense(num_classes, activation='softmax')(x)
#     model = Model(inputs=base_model.input, outputs=outputs)
#     return model

# # Create models
# base_model1 = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
# base_model2 = DenseNet201(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
# base_model3 = Xception(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# for layer in base_model1.layers:
#     layer.trainable = True

# for layer in base_model2.layers:
#     layer.trainable = True

# for layer in base_model3.layers:
#     layer.trainable = True

# model1 = create_model(base_model1, num_classes)
# model2 = create_model(base_model2, num_classes)
# model3 = create_model(base_model3, num_classes)

# # Compile models
# model1.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# model2.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# model3.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)
checkpoint1 = ModelCheckpoint('model1_best.keras', monitor='val_loss', save_best_only=True)
checkpoint2 = ModelCheckpoint('model2_best.keras', monitor='val_loss', save_best_only=True)
checkpoint3 = ModelCheckpoint('model3_best.keras', monitor='val_loss', save_best_only=True)

# Train models
history1 = model1.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint1]
)

history2 = model2.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint2]
)

history3 = model3.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint3]
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


268/268 ━━━━━━━━━━━━━━━━━━━━ 279s 786ms/step - accuracy: 0.0348 - loss: 3.6507 - val_accuracy: 0.1229 - val_loss: 3.5062 - learning_rate: 1.0000e-05
Epoch 2/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 194s 711ms/step - accuracy: 0.1494 - loss: 3.3446 - val_accuracy: 0.3716 - val_loss: 2.6962 - learning_rate: 1.0000e-05
Epoch 3/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 189s 667ms/step - accuracy: 0.4241 - loss: 2.4327 - val_accuracy: 0.6066 - val_loss: 1.6730 - learning_rate: 1.0000e-05
Epoch 4/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 179s 658ms/step - accuracy: 0.6309 - loss: 1.5073 - val_accuracy: 0.7007 - val_loss: 1.1721 - learning_rate: 1.0000e-05
Epoch 5/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 217s 710ms/step - accuracy: 0.7482 - loss: 0.9853 - val_accuracy: 0.7456 - val_loss: 0.9162 - learning_rate: 1.0000e-05
Epoch 6/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 186s 687ms/step - accuracy: 0.8043 - loss: 0.7334 - val_accuracy: 0.7749 - val_loss: 0.7795 - learning_rate: 1.0000e-05
Epoch 7/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 207s 

In [ ]:
# # Load best models
# model1.load_weights('model1_best.h5')
# model2.load_weights('model2_best.h5')
# model3.load_weights('model3_best.h5')

# Get predictions from the base models
preds1 = model1.predict(test_generator)
preds2 = model2.predict(test_generator)
preds3 = model3.predict(test_generator)

# Concatenate predictions
stacked_preds = np.concatenate((preds1, preds2, preds3), axis=1)

# Prepare labels
test_labels = test_generator.classes
test_labels = LabelBinarizer().fit_transform(test_labels)

# Train meta-classifier
meta_classifier = LogisticRegression(max_iter=1000)
meta_classifier.fit(stacked_preds, test_labels.argmax(axis=1))

# Make predictions using the meta-classifier
final_preds = meta_classifier.predict(stacked_preds)
final_preds = LabelBinarizer().fit_transform(final_preds)

# Calculate metrics
accuracy = accuracy_score(test_labels.argmax(axis=1), final_preds.argmax(axis=1))
f1 = f1_score(test_labels.argmax(axis=1), final_preds.argmax(axis=1), average='weighted')
report = classification_report(test_labels.argmax(axis=1), final_preds.argmax(axis=1))

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Classification Report:\n{report}')